In [59]:
import numpy as np

#Now, create Black-Scholes function

def bs_call_price(S0, K, r, q, T, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S0 * np.exp(-q * T) * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)


def implied_vol_bisection_bracket(price_mkt, S0, K, r, q, T,
                                  sigma_low=1e-8, sigma_high=2.0,
                                  tol=1e-6, max_iter=100, expand_max=10):
    """
    Implied volatility via bisection with explicit bracketing:
      require f(low)*f(high) < 0, where f(sigma)=BS(sigma)-price_mkt.
    """

    # 1) Arbitrage lower bound for a call (WITH DIVIDENDS)
    intrinsic = max(S0 * np.exp(-q * T) - K * np.exp(-r * T), 0.0)
    if price_mkt < intrinsic:
        return f"FAIL: price < intrinsic (mid={price_mkt:.2f} < {intrinsic:.2f})"


    # 2) Define f(sigma)
    def f(sigma):
        return bs_call_price(S0, K, r, q, T, sigma) - price_mkt

    # 3) Bracket the root: f(low)*f(high) < 0
    f_low = f(sigma_low)
    f_high = f(sigma_high)

    # A positive product just means that we need to keep expanding the upper limit in the interval (we're not rewriting the lower limit),
    # if that product is positive for a given amount of max iterations, then the volatility is too high
    # to make sense out of it. But, how much does our code allow the higher vol to be? A = 2 to the 10 (2000%)
    # Expand upper bound if needed
    j = 0
    while (f_low * f_high > 0) and (j < expand_max):
        sigma_high *= 2.0
        f_high = f(sigma_high)
        j += 1

    # If still no bracket, give up
    if f_low * f_high > 0:
        return f"FAIL: could not bracket after {j} expansions. Vol is over: {sigma_high:.2f}"
        

    # 4) Bisection with max iterations
    low, high = sigma_low, sigma_high
    i = 0

    while (high - low > tol) and (i < max_iter):
        mid = 0.5 * (low + high)
        f_mid = f(mid)

        # Keep the interval that preserves sign change
        if f_low * f_mid <= 0:
            high = mid
            f_high = f_mid

         # if f_low * f_mid > 0:
        else:
            low = mid
            f_low = f_mid

        i += 1

    return 0.5 * (low + high)
    

#Newton Raphson 

def bs_call_vega(S0, K, r, q, T, sigma):
    d1 = (np.log(S0 / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return S0 * np.exp(-q * T) * norm.pdf(d1) * np.sqrt(T)

def implied_vol_newton_or_bisect(price_mkt, S0, K, r, q, T,
                                 sigma0=0.2, tol=1e-6, max_iter=100,
                                 vega_min=1e-8):
    sigma = sigma0

    for _ in range(max_iter):
        price = bs_call_price(S0, K, r, q, T, sigma)
        f_val = price - price_mkt

        if abs(f_val) < tol:
            return sigma

        vega = bs_call_vega(S0, K, r, q, T, sigma)
        if vega < vega_min:
            # fallback: bisection is robust when vega is tiny
            return implied_vol_bisection_bracket(price_mkt, S0, K, r, T, q=q)

        sigma = sigma - f_val / vega
        if sigma <= 0:
            sigma = 1e-8

    return np.nan

    return np.nan

